# Cox regression and hazard ratio model

Cox fa analisi di ogni variabile data e predice per ognuna l'hazard ratio, ovvero una probabilità, che se >1 indica che il rischio dell'accadere dell'evento aumenta all'aumentare del valore di quella variabile (o presenza di quella variabile in caso di booleane), mentre diminuisce se l'hazard ratio è <1. 


Input : feature vectors con età del paziente alla diagnosi, last days to follow-up, evento morte booleano, miRNA-seq vector con valori normalizzati con log e quantile.

Pipeline:
   - Scaling con **Z-scaler** su campi di età e miRNA-seq
   - Applicazione di elsatic net tramite ```scikit-survival.CoxnetSurvivalAnalysis``` da addestrare (scikit-survival at: https://scikit-survival.readthedocs.io/en/stable/user_guide/coxnet.html)
      - Applicare grid search e K-fold cross validation per capire set di parametri migliori
   - Calcolo di risk score con funzione di predict
        - possibile prevedere survival function o cumulative hazard function anche, ma necessario fare fine tuning con parametro ```fit_baselin_model=True```

Motivazioni:
   - Z-scaler per portare valori predittivi su stessa scala con varianza 1 e media 0
   - Utilizzo di Cox con penalizzazione per fare feature selection e selezionare solo miRNA con maggiore rilevanza
   - Utilizzo Elastic Net poichè Lasso-Cox normale non ottimale per due motivi: non può selezionare più features di quanti sample ci sono e in gruppo di features con alta correlazione tra loro ne sceglie a caso solo una tra queste. Elastic net risolve questi usando combinazione di l1 e l2 e rendendo più robusto

## Init

In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

In [3]:
base = os.path.basename(os.getcwd())
list = os.getcwd().split(os.sep) 
list.pop(list.index(base))
ROOT = '\\'.join(list)
print(ROOT)
DATA_PATH = os.path.join(ROOT, 'datasets\\preprocessed')

SEED = 42

C:\Users\giorg\Documents\GitHub\miRNA_to_age


In [4]:
dataset = pd.read_csv(os.path.join(DATA_PATH, 'clinical_miRNA_normalized_log.csv'))

In [5]:
print(dataset.shape)
print(dataset.columns)
# print(dataset.head())
print(type(dataset.iloc[0]['days_to_death']))

(760, 1896)
Index(['days_to_death', 'age_at_initial_pathologic_diagnosis',
       'days_to_last_followup', 'Death', 'pathologic_stage_Stage I',
       'pathologic_stage_Stage IA', 'pathologic_stage_Stage IB',
       'pathologic_stage_Stage II', 'pathologic_stage_Stage IIA',
       'pathologic_stage_Stage IIB',
       ...
       'hsa-mir-941-5', 'hsa-mir-942', 'hsa-mir-943', 'hsa-mir-944',
       'hsa-mir-95', 'hsa-mir-9500', 'hsa-mir-96', 'hsa-mir-98', 'hsa-mir-99a',
       'hsa-mir-99b'],
      dtype='object', length=1896)
<class 'numpy.float64'>


## Hyper-parameters

In [6]:
num_folds = 10
scoring = 'accuracy'

In [7]:
from sksurv.datasets import load_breast_cancer
from sksurv.preprocessing import OneHotEncoder
from sksurv.linear_model import CoxnetSurvivalAnalysis, CoxPHSurvivalAnalysis

X, y = load_breast_cancer()
Xt = OneHotEncoder().fit_transform(X)
Xt.round(2).head()

print((X.iloc[:, :-4]<0).sum().sum())

# print(Xt.select_dtypes(include=['object', 'category']).columns.tolist())
# print(Xt.var)

# alphas = 10.0 ** np.linspace(-4, 4, 50)
# coefficients = {}

# cph = CoxPHSurvivalAnalysis()
# for alpha in alphas:
#     cph.set_params(alpha=alpha)
#     cph.fit(Xt, y)
#     key = round(alpha, 5)
#     coefficients[key] = cph.coef_

# coefficients = pd.DataFrame.from_dict(coefficients).rename_axis(index="feature", columns="alpha").set_index(Xt.columns)

0


## Data

In [8]:
unique, count = np.unique(dataset['age_at_initial_pathologic_diagnosis'], return_counts=True)
to_drop = [unique[u] for u in range(len(unique)) if count[u] < 5]
print(to_drop)
print(dataset.shape)

dataset=dataset[~dataset['age_at_initial_pathologic_diagnosis'].isin(to_drop)]
print(dataset.shape)

[np.int64(26), np.int64(27), np.int64(30), np.int64(31), np.int64(32), np.int64(83), np.int64(85), np.int64(86), np.int64(88), np.int64(89)]
(760, 1896)
(739, 1896)


In [81]:
dataset.describe().loc[['min', 'max']]

,days_to_death,age_at_initial_pathologic_diagnosis,days_to_last_followup,Death,pathologic_stage_Stage I,pathologic_stage_Stage IA,pathologic_stage_Stage IB,pathologic_stage_Stage II,pathologic_stage_Stage IIA,pathologic_stage_Stage IIB,...,hsa-mir-941-5,hsa-mir-942,hsa-mir-943,hsa-mir-944,hsa-mir-95,hsa-mir-9500,hsa-mir-96,hsa-mir-98,hsa-mir-99a,hsa-mir-99b
min,-1.0,29.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-9.965784,-9.965784,-9.965784,-9.965784,-9.965784,-9.965784,-9.965784,2.914571,5.317549,11.759983
max,4456.0,90.0,6796.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,-9.965784,7.513267,0.775161,6.584363,5.177018,-9.965784,8.007995,8.624918,13.930765,17.955206


In [9]:
# y = death_event and days_to_death/last_folowup
# X = all the rest
y_cols = ['Death', 'days_to_death', 'days_to_last_followup']
X_cols = [col for col in dataset.columns if col not in y_cols]

custom_dtype = np.dtype([
    ('death', np.bool_),         # O 'bool'
    ('days', np.float64)      # O 'float'
])

y = []
for index,row in dataset[y_cols].iterrows():
    if row['Death'] == 1:
        y.append(np.array((True, row['days_to_death'].item()), dtype=custom_dtype))
    elif row['Death'] == 0:
        tuple = (False, row['days_to_last_followup'].item())
        y.append(np.array(tuple, dtype=custom_dtype)) 
y = np.array(y)

X = dataset[X_cols]
# remove columns with zero-variance
# print(X.shape)
X = X.loc[:, X.var() != 0]
# print(X.shape)


## Z-scaling

In [10]:
from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import MinMaxScaler

scaler = StandardScaler()
# minmax = MinMaxScaler(feature_range=(0,15))

scaled_X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
# minmaxed_X = pd.DataFrame(minmax.fit_transform(scaled_X), columns = scaled_X.columns)

## Data splitting

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.2, stratify=X['age_at_initial_pathologic_diagnosis'])

## K-fold

In [12]:
from sklearn.model_selection import StratifiedKFold, KFold

kfold = KFold(n_splits=num_folds, shuffle=True, random_state=SEED)
#kfold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=SEED)

## Linear MLP

In [13]:
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import mean_absolute_error, accuracy_score, r2_score

class MiRNANet_3(nn.Module):
    def __init__(self, input_dim, output_dim, dropout, start_lr=0.001, lr_decay=0.3, patience=15):
        super(MiRNANet_3, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(32, output_dim),
            nn.Softmax(-1)
        )
        self.lr_decay = lr_decay
        self.patience = patience
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=start_lr)
        self.criterion = nn.L1Loss()
        self.scheduler = ReduceLROnPlateau(
            self.optimizer,
            mode='min',        # riduce LR quando la metrica (loss) smette di diminuire
            factor=self.lr_decay,        # dimezza il learning rate
            patience=self.patience        # aspetta 10 epoche senza miglioramenti
        )
    def forward(self, x):
        return self.model(x)   
    def loop(self, train_loader, test_loader, epochs=100):
        train_losses = []
        val_losses = []
        train_accs = []
        val_accs = []
        for epoch in range(epochs):
            self.model.train()
            total_train_loss = 0
            total_train_acc = 0
            for X_batch, y_batch in train_loader:
                self.optimizer.zero_grad()
                outputs = self.model(X_batch)
                loss = self.criterion(outputs, y_batch)
                loss.backward()
                self.optimizer.step()
                total_train_loss += loss.item()
                total_train_acc += accuracy_score(np.argmax(outputs.detach().numpy(), axis=1), np.argmax(y_batch, axis=1))
            
            avg_train_loss = total_train_loss / len(train_loader)
            avg_train_acc = total_train_acc / len(train_loader)
            
            self.scheduler.step(avg_train_loss)

            train_accs.append(avg_train_acc)

            self.model.eval()
            total_val_loss = 0
            total_val_acc = 0
            with torch.no_grad():
                preds = []
                trues = []
                for X_batch, y_batch in test_loader:
                    outputs = self.model(X_batch)
                    loss = self.criterion(outputs, y_batch)
                    preds.append(outputs.numpy())
                    trues.append(y_batch.numpy())
                    total_val_loss += loss.item()
                    total_val_acc += accuracy_score(np.argmax(outputs.detach().numpy(), axis=1), np.argmax(y_batch, axis=1))
                preds = np.vstack(preds)
                trues = np.vstack(trues)
            avg_val_loss = total_val_loss / len(test_loader)
            avg_val_acc = total_val_acc / len(test_loader)

            train_losses.append(avg_train_loss)
            val_losses.append(avg_val_loss)

            val_accs.append(avg_val_acc)

            print(f"Epoch {epoch+1:03d}:\nTrain Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}\nTrain Acc: {avg_train_acc:.4f} | Val Acc: {avg_val_acc:.4f}")
            mae = mean_absolute_error(trues, preds)
            print(f"Test MAE: {mae:.2f}")

        model_res = {
            'params':self.model.parameters(),
            'train_losses':train_losses,
            'eval_losses': val_losses,
            'train_accs': train_accs,
            'val_accs':val_accs,
            'preds':preds,
            'trues':trues
        }

        return model_res

In [14]:
class MiRNANet_5(nn.Module):
    def __init__(self, input_dim, output_dim, dropout, start_lr = 0.001):
        super(MiRNANet_5, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(128, output_dim)  # <--- ultimo layer lineare
        )
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=start_lr)
        self.criterion = nn.MSELoss()
        self.scheduler = ReduceLROnPlateau(
            self.optimizer,
            mode='min',        # riduce LR quando la metrica (loss) smette di diminuire
            factor=0.6,        # dimezza il learning rate
            patience=15        # aspetta 10 epoche senza miglioramenti
        )
    def forward(self, x):
        return self.model(x)
    
    def loop(self, train_loader, test_loader, epochs=100):
        train_losses = []
        val_losses = []
        for epoch in range(epochs):
            self.model.train()
            total_train_loss = 0
            for X_batch, y_batch in train_loader:
                self.optimizer.zero_grad()
                outputs = self.model(X_batch)
                # print(outputs)
                loss = self.criterion(outputs, y_batch)
                loss.backward()
                self.optimizer.step()
                total_train_loss += loss.item()
            
            avg_train_loss = total_train_loss / len(train_loader)
            self.scheduler.step(avg_train_loss)

            self.model.eval()
            total_val_loss = 0
            with torch.no_grad():
                preds = []
                trues = []
                for X_batch, y_batch in test_loader:
                    outputs = self.model(X_batch)
                    loss = self.criterion(outputs, y_batch)
                    preds.append(outputs.numpy())
                    trues.append(y_batch.numpy())
                preds = np.vstack(preds)
                trues = np.vstack(trues)
                total_val_loss += loss.item()
            avg_val_loss = total_val_loss / len(test_loader)

            train_losses.append(avg_train_loss)
            val_losses.append(avg_val_loss)

            print(f"Epoch {epoch+1:03d} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

        model_res = {
            'params':self.model.parameters(),
            'train_losses':train_losses,
            'eval_losses': val_losses
        }

        return model_res

In [15]:
import torch

y_train_fixed = np.array([(row['death'], row['days']) for row in y_train], dtype=[('death', np.bool_), ('days', np.float64)])
y_test_fixed = np.array([(row['death'], row['days']) for row in y_test], dtype=[('death', np.bool_), ('days', np.float64)])

X_train_mlp = torch.tensor(X_train.values, dtype=torch.float32)
X_test_mlp = torch.tensor(X_train.values, dtype=torch.float32)
y_train_mlp = torch.tensor([row for row in y_train_fixed], dtype=torch.float32)
y_test_mlp = torch.tensor([row for row in y_test_fixed], dtype=torch.float32)

In [ ]:
from skorch import NeuralNetRegressor
from sklearn.model_selection import GridSearchCV
from skorch.callbacks import LRScheduler


# GRID SEARCH
net = NeuralNetRegressor(
    module=MiRNANet_3,
    module__input_dim=X_train_mlp.shape[1],
    module__output_dim=y_train_mlp.shape[1],
    max_epochs=60,
    lr=1e-3,
    batch_size=32,
    optimizer=torch.optim.Adam,
    optimizer__weight_decay=1e-5,
    optimizer__lr=0.001,
    criterion=nn.L1Loss(),
    callbacks=[
         ('lr_scheduler', LRScheduler(
             ReduceLROnPlateau, 
             mode='min', 
             factor=0.7, 
             patience=15, 
             monitor='valid_loss',
             min_lr=1e-6
         ))
    ],
)

params = {
    #'module__hidden1': [128, 256, 512],
    #'module__hidden2': [64, 128, 256],
    'module__dropout': [0.2, 0.3, 0.5],
    'module__start_lr': [1e-3, 1e-4],
    #'module__lr_decay': [0.3, 0.4],
    #'optimizer__weight_decay': [0, 1e-5, 1e-4],
    #'callbacks__lr_scheduler__factor': [0.8, 0.6, 0.4],
    #'callbacks__lr_scheduler__patience': [15, 10, 5],
    'lr': [1e-1, 5e-4]
}

gs = GridSearchCV(net, 
                  params, 
                  refit=True, 
                  cv=kfold,  # cross-validation folds 
                  scoring='neg_mean_squared_error', 
                  verbose=1)
gs.fit(X_train_mlp, y_train_mlp)

best_model = gs.best_estimator_
print("Best hyperparameters:", gs.best_params_)

Fitting 10 folds for each of 12 candidates, totalling 120 fits
  epoch    train_loss    valid_loss      lr     dur
-------  ------------  ------------  ------  ------
      1      368.6932      441.1831  0.0010  0.1078
      2      368.5039      441.0685  0.0010  0.1167
      3      368.4366      441.0355  0.0010  0.1131
      4      368.4205      441.0250  0.0010  0.1193
      5      368.4160      441.0207  0.0010  0.1057
      6      368.4152      441.0184  0.0010  0.1133
      7      368.4145      441.0170  0.0010  0.0839
      8      368.4143      441.0161  0.0010  0.0776
      9      368.4140      441.0154  0.0010  0.0999
     10      368.4140      441.0146  0.0010  0.0980
     11      368.4135      441.0140  0.0010  0.0871
     12      368.4134      441.0137  0.0010  0.0988
     13      368.4134      441.0133  0.0010  0.0767
     14      368.4135      441.0130  0.0010  0.0810
     15      368.4133      441.0127  0.0010  0.1039
     16      368.4132      441.0125  0.0010  0.0980
 

In [ ]:
preds = best_model.predict(X_test_mlp)
mae = mean_absolute_error(y_test_mlp, preds)
r2 = r2_score(y_test_mlp, preds)
print(f"Test MAE: {mae:.2f} | R2: {r2:.2f}")

## Elastic net (Lasso-Cox)

In [89]:
def plot_coefficients(coefs, n_highlight):
    _, ax = plt.subplots(figsize=(9, 6))
    alphas = coefs.columns
    for row in coefs.itertuples():
        ax.semilogx(alphas, row[1:], ".-", label=row.Index)

    alpha_min = alphas.min()
    top_coefs = coefs.loc[:, alpha_min].map(abs).sort_values().tail(n_highlight)
    for name in top_coefs.index:
        coef = coefs.loc[name, alpha_min]
        plt.text(alpha_min, coef, name + "   ", horizontalalignment="right", verticalalignment="center")

    ax.yaxis.set_label_position("right")
    ax.yaxis.tick_right()
    ax.grid(True)
    ax.set_xlabel("alpha")
    ax.set_ylabel("coefficient")

In [90]:
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sklearn.model_selection import cross_val_score

alphas = 10.0 ** np.linspace(-2, 2, 20)
coefficients = {}

print(alphas)

cph = CoxPHSurvivalAnalysis()
for alpha in alphas:
    cph.set_params(alpha=alpha)
    cph.fit(X_train, y_train)
    key = round(alpha, 5)
    coefficients[key] = cph.coef_
    print(f"Finished fitting for alpha coefficient : {alpha}")

coefficients = pd.DataFrame.from_dict(coefficients).rename_axis(index="feature", columns="alpha").set_index(X.columns)

[1.00000000e-02 1.62377674e-02 2.63665090e-02 4.28133240e-02
 6.95192796e-02 1.12883789e-01 1.83298071e-01 2.97635144e-01
 4.83293024e-01 7.84759970e-01 1.27427499e+00 2.06913808e+00
 3.35981829e+00 5.45559478e+00 8.85866790e+00 1.43844989e+01
 2.33572147e+01 3.79269019e+01 6.15848211e+01 1.00000000e+02]


c:\Users\Matteo Bulgarelli\anaconda3\envs\tirocinio-venv\Lib\site-packages\sksurv\linear_model\coxph.py:200: RuntimeWarning: overflow encountered in exp
  risk_set += np.exp(xw[k])
c:\Users\Matteo Bulgarelli\anaconda3\envs\tirocinio-venv\Lib\site-packages\sksurv\linear_model\coxph.py:197: RuntimeWarning: overflow encountered in exp
  risk_set2 += np.exp(xw[k])


Finished fitting for alpha coefficient : 0.01


c:\Users\Matteo Bulgarelli\anaconda3\envs\tirocinio-venv\Lib\site-packages\sksurv\linear_model\coxph.py:200: RuntimeWarning: overflow encountered in exp
  risk_set += np.exp(xw[k])
c:\Users\Matteo Bulgarelli\anaconda3\envs\tirocinio-venv\Lib\site-packages\sksurv\linear_model\coxph.py:197: RuntimeWarning: overflow encountered in exp
  risk_set2 += np.exp(xw[k])


Finished fitting for alpha coefficient : 0.016237767391887217


...

Finished fitting for alpha coefficient : 0.7847599703514611


c:\Users\Matteo Bulgarelli\anaconda3\envs\tirocinio-venv\Lib\site-packages\sksurv\linear_model\coxph.py:197: RuntimeWarning: overflow encountered in exp
  risk_set2 += np.exp(xw[k])


Finished fitting for alpha coefficient : 1.2742749857031335


c:\Users\Matteo Bulgarelli\anaconda3\envs\tirocinio-venv\Lib\site-packages\sksurv\linear_model\coxph.py:197: RuntimeWarning: overflow encountered in exp
  risk_set2 += np.exp(xw[k])


Finished fitting for alpha coefficient : 2.06913808111479
Finished fitting for alpha coefficient : 3.359818286283781
Finished fitting for alpha coefficient : 5.455594781168514
Finished fitting for alpha coefficient : 8.858667904100823
Finished fitting for alpha coefficient : 14.38449888287663
Finished fitting for alpha coefficient : 23.357214690901213
Finished fitting for alpha coefficient : 37.92690190732246
Finished fitting for alpha coefficient : 61.584821106602604
Finished fitting for alpha coefficient : 100.0


In [91]:
coefficients.to_csv(os.path.join(DATA_PATH, 'Cox_coefficients.csv'), index=False)

In [ ]:
cv = KFold(n_splits=5, shuffle=True, random_state=0)
gcv = GridSearchCV(
    make_pipeline(StandardScaler(), CoxnetSurvivalAnalysis(l1_ratio=0.9)),
    param_grid={"coxnetsurvivalanalysis__alphas": [[v] for v in map(float, estimated_alphas)]},
    cv=cv,
    error_score=0.5,
    n_jobs=1,
).fit(Xt, y)

cv_results = pd.DataFrame(gcv.cv_results_)
